In [152]:
import pandas as pd
import datetime
import numpy as np

### Indexing

To demonstrate the different types of indexing available in `pandas` we'll create a test dataframe
that contains a mix of different keys and values. We use a mix of value types to demonstrate some
common pitfalls with the methods available. Hopefully it will help clarify when/if you should
use the different indexing approaches!

In [153]:
# Create a demo dataframe
df = pd.DataFrame(
    np.random.randint(0, 1000, (6,3)),
    columns=['AAPL','GOOG','MSFT'],
    index=['1', 8, 3, 4, 'five', 6]
)
        
df

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
five,139,654,261
6,314,940,937


### Standard indexing

The default behaviour in `pandas` is to index along the column (horizontal) axis, when 
using `df[]` style syntax. This is confusing if you're used to `numpy` since this works the other way around. This type of indexing is also quite limited. Use this when you want to select a subset of columns.

The matching is done by column values (labels), not position.

For example:

In [154]:
df['AAPL']

1        11
8       859
3       339
4       897
five    139
6       314
Name: AAPL, dtype: int64

In [155]:
df[ ['AAPL','GOOG'] ]  # Pass in a list of column labels to select multiple columns

,AAPL,GOOG
1,11,859
8,859,766
3,339,820
4,897,628
five,139,654
6,314,940


In [156]:
df[ ['GOOG','AAPL'] ]  # Reorder columns

,GOOG,AAPL
1,859,11
8,766,859
3,820,339
4,628,897
five,654,139
6,940,314


In [ ]:
# df[1] # ERROR: You can't access columns by position using this syntax

In [ ]:
# df['AAPL', 1] # ERROR: You can't access the rows through this syntax!

### .loc indexing

`.loc` indexing is similar to `df[]` style indexing in that it uses labels. However, it allows
selection along multiple axes, and selects along the row axis first (like numpy).

As for `df[]` syntax it matches on the basis of label (or value) — think "**l**oc for **l**abel". It does not match by position.

In [160]:
df.loc['1'] # Remember our first index value was a string

AAPL     11
GOOG    859
MSFT    245
Name: 1, dtype: int64

In [ ]:
# df.loc[1] # ERROR: There is no 1 int value in the index, and .loc doesnt match position

In [162]:
df.loc[3,'AAPL'] # Index on both axes, starting with the vertical (row) axis like numpy

339

In [163]:
df.loc[3, ['AAPL','GOOG'] ] # You can still access multiple indices in any axis using lists

AAPL    339
GOOG    820
Name: 3, dtype: int64

In [164]:
df.loc[[3,'five'], ['AAPL','GOOG'] ] # Again

,AAPL,GOOG
3,339,820
five,139,654


### .iloc indexing 

This is `numpy`-like indexing, using the zero-indexed position of the row/column to select 
the value. As with `.loc` this works on the row axis (vertical) first, like numpy. The values
(labels) in the index are ignored. Think ""**i**loc` for **i**ndex"".

In [165]:
df.iloc[1] # Accessing by position

AAPL    859
GOOG    766
MSFT    605
Name: 8, dtype: int64

In [166]:
df.iloc[2] # Again

AAPL    339
GOOG    820
MSFT    318
Name: 3, dtype: int64

In [167]:
df.iloc[-1] # Can use reverse (-1 = last in axis)

AAPL    314
GOOG    940
MSFT    937
Name: 6, dtype: int64

In [168]:
df.iloc[[0,1,2]] # Can access by list

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318


In [169]:
df.iloc[[3,2,3]] # Can reorder (and duplicate!) by list

,AAPL,GOOG,MSFT
4,897,628,855
3,339,820,318
4,897,628,855


In [170]:
df.iloc[[0,1],[0,1]] # Subset along multiple axes

,AAPL,GOOG
1,11,859
8,859,766


In [171]:
df.iloc[0:3] # Slicing

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318


In [ ]:
# df.iloc['1'] # ERROR: Labels won't work

### .ix indexing

This is a weird mix of both `.iloc` and `.loc` indexing. It matches again in numpy order, however it attempts to match on the basis of 
the label, and if that fails, then tries to match by position — *unless* the index is entirely
integer. If that sounds like a nightmare, it is. This has some uses for complex indexing into 
multilevel indices when you use them. But I would avoid it generally — see below for why:
    
Think ".**ix** for **I** shouldn't use this unless I'm e**x**ceptionally desperate".

In [173]:
df.ix['1']

AAPL     11
GOOG    859
MSFT    245
Name: 1, dtype: int64

In [174]:
df.ix[1] # This is selecting row 1, which isn't in (as an int) so misses and selects by numeric row

AAPL    859
GOOG    766
MSFT    605
Name: 8, dtype: int64

In [175]:
df.ix[1,'AAPL'] # Multi axis indexing works

859

In [176]:
df.ix[['1', 2, 3]]  # So does list-based indexing, but now it won't auto-match the second value

,AAPL,GOOG,MSFT
1,11.0,859.0,245.0
2,NaN,NaN,NaN
3,339.0,820.0,318.0


In [ ]:
# df.ix[[0,1,2]] # ERROR: Cannot list-access by position 

### Boolean indexing

Sometimes you want to apply a filter to determine whether to keeping something in your table
or not. You can do this using a boolean index. These are supported by all of `df[]`, `.iloc`, `.loc` and `.ix` (it shouldn't matter which you use).

In [178]:
df

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
five,139,654,261
6,314,940,937


In [179]:
# Mask to determine whether the value in the index is numeric
rmask = [str(v).isdigit() for v in df.index.values]
rmask

[True, True, True, True, False, True]

In [180]:
df[rmask]

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
6,314,940,937


In [181]:
df.iloc[rmask]

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
6,314,940,937


In [182]:
df.loc[rmask]

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
6,314,940,937


In [183]:
df.ix[rmask]

,AAPL,GOOG,MSFT
1,11,859,245
8,859,766,605
3,339,820,318
4,897,628,855
6,314,940,937


The only real issue here is if you try and index on both the rows and column axis at the same time because you can't do that with `df[]` syntax. All the others are fine. For example -

In [184]:
# Test if the index values are numeric and less than 4
rmask = [str(v).isdigit() and int(v) < 4 for v in df.index.values]
rmask

[True, False, True, False, False, False]

In [185]:
cmask = [True, False, True]

In [ ]:
# df[rmask, cmask]  # ERROR: Can't index in two axes

In [187]:
df.iloc[rmask, cmask]

,AAPL,MSFT
1,11,245
3,339,318


In [188]:
df.loc[rmask, cmask]

,AAPL,MSFT
1,11,245
3,339,318


In [189]:
df.ix[rmask, cmask]

,AAPL,MSFT
1,11,245
3,339,318


### Working with datetimes in Pandas

Below we create an example `DataFrame` passing a `numpy` array of zeros, and a set of columns and index values. Here we're using strings for the index values.

In [190]:
# Create a demo dataframe
df = pd.DataFrame(
    np.zeros((3,3)),
    columns=['AAPL','GOOG','MSFT'],
    index=['2015-05-01','2015-05-02','2015-05-03']
)
        
df

,AAPL,GOOG,MSFT
2015-05-01,0.0,0.0,0.0
2015-05-02,0.0,0.0,0.0
2015-05-03,0.0,0.0,0.0


There is a neat utility functon in `pandas` called `.to_datetime` that can be used 
to convert from a string representation of a time
to a Python `datetime` object. For example:

In [191]:
pd.to_datetime('2011-06-15')

Timestamp('2011-06-15 00:00:00')

We could use this to create the index with all datetime objects, as follows:

In [192]:
# Create a demo dataframe
df = pd.DataFrame(
    np.zeros((3,3)),
    columns=['AAPL','GOOG','MSFT'],
    index=[
        pd.to_datetime('2015-05-01'),
        pd.to_datetime('2015-05-02'),
        pd.to_datetime('2015-05-03'),
        ]
)
        
df

,AAPL,GOOG,MSFT
2015-05-01,0.0,0.0,0.0
2015-05-02,0.0,0.0,0.0
2015-05-03,0.0,0.0,0.0


They look the same (hence our problems!) but you can see the type of the values in the index 
by looking at it directly. For example:

In [193]:
df.index

DatetimeIndex(['2015-05-01', '2015-05-02', '2015-05-03'], dtype='datetime64[ns]', freq=None)